In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
from torch.utils.data import DataLoader
import torch
from torch.autograd import Variable
from faster_rcnn.utils.cython_bbox import bbox_overlaps
from pycrayon import CrayonClient

import cPickle
from torch.optim import SGD, RMSprop, Adam
from torch.optim.lr_scheduler import StepLR
from datetime import datetime
from faster_rcnn.utils.datasets.adapter import convert_data
from faster_rcnn.utils.evaluate.metter import AverageMeter
from faster_rcnn.utils.display.images import imshow, result_show

In [3]:
with open('sorted_index', 'rb') as fp:
    sorted_index = cPickle.load(fp)

### Đọc dữ liệu từ MS COCO dataset


In [4]:
import os
import torchvision.transforms as transforms
from faster_rcnn.utils.datasets.mscoco.dataset import CocoData
from faster_rcnn.utils.datasets.data_generator import CocoGenerator
from faster_rcnn.utils.datasets.data_generator import Enqueuer

dataDir = './data/mscoco'
dataType = 'train2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
batch_size = 6

images_dir = os.path.join(dataDir,'images', dataType)
cap = CocoData(root = images_dir, annFile = annFile)

data_gen = CocoGenerator(data=cap, sorted_index=sorted_index, batch_size=batch_size)
queue = Enqueuer(generator=data_gen)
queue.start(max_queue_size=10, workers=2)
train_data_generator = queue.get()


loading annotations into memory...
Done (t=10.45s)
creating index...
index created!


INFO:faster_rcnn.utils.datasets.mscoco.dataset:need more than 0 values to unpack


In [5]:
dataDir = './data/mscoco'
dataType = 'val2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
batch_size = 6

images_dir = os.path.join(dataDir,'images', dataType)
val_cap = CocoData(root = images_dir, annFile = annFile)

val_data_gen = CocoGenerator(data=val_cap, batch_size=batch_size, shuffle=True, seed=2)
val_queue = Enqueuer(generator=val_data_gen)
val_queue.start(max_queue_size=10, workers=2)
val_data_generator = val_queue.get()


loading annotations into memory...
Done (t=6.59s)
creating index...
index created!


INFO:faster_rcnn.utils.datasets.mscoco.dataset:need more than 0 values to unpack


Thử hiển thị ảnh cùng các bounding boxes

In [6]:
from faster_rcnn.faster_rcnn import FastRCNN


### Tính toán feed-forward


Chúng ta sử dụng một ảnh có kích thước đầu vào là  `(width , height) = (600, 800)`

Input:
    - im_data : 
        kích thước : (batch_size, dim, witdh, height)
    - ground_boxes: 
        kích thước (n, 4)
        

In [7]:
categories = ['__background__'] + [x['name'] for x in cap.coco.loadCats(cap.coco.getCatIds())]

In [8]:
net = FastRCNN(categories, debug=False)
net.cuda()
net.train()

FastRCNN(
  (rpn): RPN(
    (features): DataParallel(
      (module): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace)
        (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (15): 

In [9]:
param = filter(lambda x: x.requires_grad, net.parameters())
optimizer = SGD(param, lr=1e-3, momentum=0.9, weight_decay=0.0005)
exp_lr_scheduler = StepLR(optimizer, step_size=1000, gamma=0.95)

In [10]:
def evaluate(data_gen ,model, steps_per_epoch=2000, epochs=1):
    model.eval()
    val_loss = AverageMeter()
    val_cross_entropy = AverageMeter()
    val_loss_box = AverageMeter()
    val_rpn_loss = AverageMeter()
    
    for epoch in range(epochs):
        for step in range(1, steps_per_epoch +1):
            blobs = data_gen.next()
            batch_tensor, im_info, batch_boxes, batch_boxes_index = convert_data(blobs)
            cls_prob, bbox_pred, rois = model(batch_tensor, im_info, batch_boxes, batch_boxes_index)

            loss = model.loss
            
            val_loss_box.update(model.loss_box.item())
            val_cross_entropy.update(model.cross_entropy.item())
            val_loss.update(loss.item())
            val_rpn_loss.update(model.rpn.loss.item())
    
    log_text = 'val_loss: %.4f' % (val_loss.avg)
    print(log_text)
    return val_loss ,val_cross_entropy, val_loss_box, val_rpn_loss


In [11]:
def train(train_data_gen, val_data_gen, optimizer, lr_scheduler ,model, epochs, steps_per_epoch, val_step_per_epoch):
    
    exp_name = datetime.now().strftime('vgg16_%m-%d_%H-%M')
    cc = CrayonClient(hostname="crayon", port=8889)
    exp = cc.create_experiment(exp_name)
    
    
    train_loss = AverageMeter()
    cross_entropy = AverageMeter()
    loss_box = AverageMeter()
    rpn_loss = AverageMeter()
    current_step = 0
    
    for epoch in range(epochs):
        model.train()
        for step in range(1, steps_per_epoch +1):
            lr_scheduler.step()        
            blobs = data_gen.next()
            batch_tensor, im_info, batch_boxes, batch_boxes_index = convert_data(blobs)

            cls_prob, bbox_pred, rois = model(batch_tensor, im_info, batch_boxes, batch_boxes_index)
            cls_data = cls_prob.data.cpu().numpy()
            max_class_idx = np.argmax(cls_data, axis=1)
            loss = model.loss
            cross_entropy.update(model.cross_entropy.item())
            loss_box.update(model.loss_box.item())
            train_loss.update(loss.item())
            rpn_loss.update(model.rpn.loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            current_step = epoch * steps_per_epoch + step
            if step % 20 == 1:
                log_text = 'epoch: %d : step %d,  loss: %.4f at %s' % (
                    epoch + 1, step , train_loss.avg, datetime.now().strftime('%m\%d_%H:%M'))
                print(log_text)

            if step % 20 == 0:
                exp.add_scalar_value('train_loss', train_loss.avg, step=current_step)
                exp.add_scalar_value('rpn_loss', rpn_loss.avg, step=current_step)
                exp.add_scalar_value('cross_entropy', cross_entropy.avg, step=current_step)
                exp.add_scalar_value('loss_box', loss_box.avg, step=current_step)


                
        torch.save(model.state_dict(), './checkpoints/faster_model_at_epoch_%s.pkl' % (epoch + 1)) 
        val_loss ,val_cross_entropy, val_loss_box, val_rpn_loss = evaluate(val_data_gen, model, val_step_per_epoch)
        exp.add_scalar_value('val_loss', val_loss.avg, step=current_step)
        exp.add_scalar_value('val_rpn_loss', val_rpn_loss.avg, step=current_step)
        exp.add_scalar_value('val_cross_entropy', val_cross_entropy.avg, step=current_step)
        exp.add_scalar_value('val_loss_box', val_loss_box.avg, step=current_step)

In [ ]:
train(train_data_generator, val_data_generator ,optimizer=optimizer,lr_scheduler=exp_lr_scheduler, model=net, epochs=200, steps_per_epoch=6000, val_step_per_epoch=1000)

faster_rcnn/rpn_msr/anchor_target_layer.py:202: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  current_batch_boxes = batch_boxes[[batch_boxes_index == i]]
faster_rcnn/rpn_msr/proposal_target_layer.py:75: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  current_gt_boxes = gt_boxes[[gt_boxes_index == i]]
faster_rcnn/faster_rcnn.py:169: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


epoch: 1 : step 1,  loss: 10.8467 at 08\24_04:48
epoch: 1 : step 21,  loss: 6.4949 at 08\24_04:49
epoch: 1 : step 41,  loss: 5.6954 at 08\24_04:49


INFO:faster_rcnn.utils.datasets.mscoco.dataset:need more than 0 values to unpack


epoch: 1 : step 61,  loss: 5.1289 at 08\24_04:50


INFO:faster_rcnn.utils.datasets.mscoco.dataset:need more than 0 values to unpack


...

INFO:faster_rcnn.utils.datasets.mscoco.dataset:need more than 0 values to unpack


epoch: 1 : step 2141,  loss: 2.9760 at 08\24_05:42


INFO:faster_rcnn.utils.datasets.mscoco.dataset:need more than 0 values to unpack


epoch: 1 : step 2161,  loss: 2.9751 at 08\24_05:43


INFO:faster_rcnn.utils.datasets.mscoco.dataset:need more than 0 values to unpack


In [ ]:
net.load_state_dict(torch.load('faster.pkl'))

In [ ]:
pred_boxes, scores, classes, rois, im_data = net.detect("./test7.jpg", thr=0.5)
print classes
print scores
print pred_boxes
imshow(im_data[0],[] ,pred_boxes, random=True)

In [13]:
print 1

1


In [14]:
__

''